# Imports

In [ ]:
from bs4 import BeautifulSoup as Soup
import pandas as pd

In [2]:
import re
import os
import time
import copy

# Work with data

In [43]:
directory = "books/All_books_wt_folders"
files = os.listdir(directory)
print(f'Количество книг в папке: {len(files)}')
print(files)

Количество книг в папке: 83
['Возвращение в Форност - Дмитрий Володихин, Сергей Петров.fb2', '09. Баллада об Аотру и Итрун.fb2', 'Маккирнан. Железная башня.fb2', 'Возвращение-Бьортнота_-сына-Бьортхельма.fb2', '2. Хоббит и Саруман (2002) Дмитрий Суслин.fb2', 'Сапковский - Доля правды (пер. Гладысь).fb2', 'Bruks_Shannara_0_Pervyiy_korol_Shannaryi.fb2', 'Сапковский 08 Что-то кончится, что-то начнется (пер. Брюн).fb2', '04. Фермер Джайлз из Хэма.fb2', 'Dzhordan_Koleso_Vremeni_4_Voshodyaschaya_ten.72384.fb2', 'Сапковский 1 Башня шутов (пер. Вайсброт).fb2', '2. Хоббит, который познал истину (2003) Вадим Проскурин.fb2', 'Роверандом.fb2', 'Кузнец-из-Большого-Вуттона.fb2', 'Bruks_Shannara_4_Potomki_Shannaryi.fb2', 'Bruks_Shannara_1_Mech_Shannaryi.fb2', '1. Эльфийский клинок (1993) Ник Перумов.fb2', '1. Оруженосец - Олег Верещагин.fb2', 'Сильмариллион.fb2', 'Лист-работы-Мелкина.fb2', '01. Хоббит, или Туда и обратно.fb2', 'Dzhordan_Koleso_Vremeni_2_Velikaya_Ohota.178494.fb2', 'Сапковский - Ведьма

In [44]:
datas_list = []
def book_to_data(book_path):
    encodings = ['utf-8', 'cp1252', 'Windows-1251', 'ISO-8859-5', 'KOI8-R']
    for enc in encodings:
        with open(book_path, 'r', encoding=enc) as xml:
            try:
                soup = Soup(xml.read(), 'xml')
                check = soup.find('p').text
                if not re.search('[А-Яа-я]', check):
                    continue
                else:
                    break
            except:
                print(f'Не смог сделать книгу {book_path} в кодировке {enc}')
                continue
    
    author = soup.find('author').text
    book_title = soup.find('book-title').text
    print(f'Смог сделать книгу {book_title} в кодировке {enc}')
    pieces = soup.find_all('section')
    book_data = []

    for piece in pieces:
        title = piece.find('title')
        if not title:
            title = 'no_title'
        else:
            title = title.text
        description = piece.find_all('p')[1:]
        book_path_new = book_path.split('/')[-1]
        new_data = pd.DataFrame({'author': [author], 'book_title': [book_title],
                                 'name': [title], 'description': [description], 
                                 'book_path': [book_path_new]})
        book_data.append(new_data)
        
    if len(book_data):
        datas_list.extend(book_data)
    else:
        print(f"Error for {book_title}")
    print('done')

In [ ]:
datas_list = []
for file in files:
    path = os.path.join(directory, file)
    print('\n' ,'='*20, '\n')

    book_to_data(path)

In [46]:
data = pd.concat(datas_list)
data_copy = data.copy()

In [47]:
data_copy.loc[data_copy.book_path == 'Братство-кольца.fb2']

author book_title      name  \
0   word    Unknown  no_title   

                                         description            book_path  
0  [[ЛЕТОПИСЬ ПЕРВАЯ], [“ХРАНИТЕЛИ”], [Джон Ронал...  Братство-кольца.fb2

In [48]:
if len(files) == data.book_title.unique().shape[0]:
    print('Все книги успешно добавлены')
else:
    print('Что-то пошло не так')
    print(f'уникальных имен: {data.book_title.unique().shape[0]}, \nДолжно быть книг: {len(files)}')

Что-то пошло не так
уникальных имен: 82, 
Должно быть книг: 83


## Clear doc data

In [49]:
exported_from_doc = []
dir_docs = '/home/darya/Загрузки/Толкин/Произведения Толкина'
exported_from_doc = os.listdir(dir_docs)
print(exported_from_doc)
# Поменять на fb2, добавить дефисы вместо пробела, сделать диктом тайтл: путь

['Дерево и лист.doc', 'Две башни.doc', 'Сильмариллион.doc', 'Хоббит или туда и обратно.doc', 'Амбарканта.doc', 'Дети Хурина.doc', 'Возвращение короля.doc', 'Роверандом.doc', 'Братство кольца.doc', 'Возвращение Бьортнота, сына Бьортхельма.doc', 'Кузнец из Большого Вуттона.doc', 'Лист работы Мелкина.doc']


In [50]:
new_data_list = []
for path in exported_from_doc:
    book_title = path.rstrip('.doc') 
    book_path = '-'.join(book_title.split()) + ('.fb2')
    new_data_list.append(pd.DataFrame({'book_title': [book_title], 'book_path': [book_path]}))

new_data_list = pd.concat(new_data_list)
new_data_list['from_doc'] = 1 #Проставлю единицы книгам, которые были переделаны из doc в fb2 на стороннем сайте. Им нужна доп очистка.

In [51]:
new_data_list

book_title  \
0                            Дерево и лист   
0                                Две башни   
0                            Сильмариллион   
0                Хоббит или туда и обратно   
0                               Амбарканта   
0                              Дети Хурина   
0                       Возвращение короля   
0                               Роверандом   
0                          Братство кольца   
0  Возвращение Бьортнота, сына Бьортхельма   
0               Кузнец из Большого Вуттона   
0                      Лист работы Мелкина   

                                     book_path  from_doc  
0                            Дерево-и-лист.fb2         1  
0                                Две-башни.fb2         1  
0                            Сильмариллион.fb2         1  
0                Хоббит-или-туда-и-обратно.fb2         1  
0                               Амбарканта.fb2         1  
0                              Дети-Хурина.fb2         1  
0                       Возвращение-короля.fb2         1  
0                               Роверандом.fb2         1  
0                          Братство-кольца.fb2         1  
0  Возвращение-Бьортнота,-сына-Бьортхельма.fb2         1  
0               Кузнец-из-Большого-Вуттона.fb2         1  
0                      Лист-работы-Мелкина.fb2         1

In [52]:
book_path_mapping = new_data_list.set_index('book_path')
book_path_mapping

book_title  \
book_path                                                                              
Дерево-и-лист.fb2                                                      Дерево и лист   
Две-башни.fb2                                                              Две башни   
Сильмариллион.fb2                                                      Сильмариллион   
Хоббит-или-туда-и-обратно.fb2                              Хоббит или туда и обратно   
Амбарканта.fb2                                                            Амбарканта   
Дети-Хурина.fb2                                                          Дети Хурина   
Возвращение-короля.fb2                                            Возвращение короля   
Роверандом.fb2                                                            Роверандом   
Братство-кольца.fb2                                                  Братство кольца   
Возвращение-Бьортнота,-сына-Бьортхельма.fb2  Возвращение Бьортнота, сына Бьортхельма   
Кузнец-из-Большого-Вуттона.fb2                            Кузнец из Большого Вуттона   
Лист-работы-Мелкина.fb2                                          Лист работы Мелкина   

                                             from_doc  
book_path                                              
Дерево-и-лист.fb2                                   1  
Две-башни.fb2                                       1  
Сильмариллион.fb2                                   1  
Хоббит-или-туда-и-обратно.fb2                       1  
Амбарканта.fb2                                      1  
Дети-Хурина.fb2                                     1  
Возвращение-короля.fb2                              1  
Роверандом.fb2                                      1  
Братство-кольца.fb2                                 1  
Возвращение-Бьортнота,-сына-Бьортхельма.fb2         1  
Кузнец-из-Большого-Вуттона.fb2                      1  
Лист-работы-Мелкина.fb2                             1

In [53]:
data['book_title_new'] = data['book_path'].map(book_path_mapping['book_title'])
data['from_doc'] = data['book_path'].map(book_path_mapping['from_doc'])

In [54]:
data.from_doc = data.from_doc.fillna(0).astype('int32')

In [55]:
data.loc[data.from_doc == 1, 'book_title'] = data.loc[data.from_doc == 1, 'book_title_new']

In [56]:
doc_datas = data.loc[data.from_doc == 1]

In [ ]:
doc_datas.author = 'Джон Рональд Руэл ТОЛКИН'

In [ ]:
doc_datas.description = doc_datas.description.apply(lambda x: x[1:])

In [59]:
loc_tale = doc_datas.book_title.isin(['Дети Хурина', 'Роверандом', 'Сильмариллион',]) #небольшие книги, для них достаточно убрать меньшее кол-во слов в описании
doc_datas.loc[loc_tale, 'description'] = doc_datas.loc[loc_tale, 'description'].apply(lambda x: x[1:])

In [60]:
loc_novel = doc_datas.book_title.isin(['Возвращение короля', 'Две башни', 'Братство кольца']) #романы, здесь нужно убрать больше слов
doc_datas.loc[loc_novel, 'description'] = doc_datas.loc[loc_novel, 'description'].apply(lambda x: x[3:])

In [ ]:
# Разделение на главы с помощью эвристик. (либо форматирование, либо цифры, либо капс)

In [ ]:
books = list(doc_datas.book_title)
template = {'author': 'Джон Рональд Руэл ТОЛКИН','book_title': '', 'name': '', 'description': [], 'from_doc': 1}
totle_structure = []
for book in books:
    this_chapter_data = None
    for t in doc_datas.loc[doc_datas['book_title'] == book].description:
        for txt in t:
            txt_str = str(txt)
            txt_text = txt.text
            if (('<strong>' in txt_str) or txt_text.lstrip('123456789.').isupper() or txt_text.isdigit()) and not bool(re.search(r"[!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]", txt_text)):
                print(txt_str, book)
                if this_chapter_data:
                    totle_structure.append(this_chapter_data)
                this_chapter_data = copy.deepcopy(template)
                this_chapter_data['book_title'] = book
                title = txt_text 
                print(f'title = {title}\n ' )
                this_chapter_data['name'] = title
            else:
                # print(txt)
                if not this_chapter_data:
                    continue
                if txt:
                    this_chapter_data['description'].append(txt)
        

In [109]:
doc_datas_chaptered = pd.DataFrame(totle_structure)

In [110]:
new_data = pd.concat([data_copy, doc_datas_chaptered])

# Clear books descriptions


In [ ]:
list(new_data.name)

In [114]:
new_data.description = new_data.description.apply(lambda x: '. '.join([y.text for y in x]))

In [116]:
print(list(new_data.description.sample(1)))
print(len(new_data))

['Иларх — командующий илой, подразделением кавалерии.']
4062


In [117]:
data.description.str.len().describe()

count    3982.000000
mean      103.398543
std       359.604130
min         0.000000
25%         1.000000
50%         2.000000
75%       107.000000
max      9233.000000
Name: description, dtype: float64

In [119]:
list(new_data.loc[(new_data.description.str.len()> 300) & (new_data.description.str.len() < 600), 'description'].sample(10, replace=True))

['В конце Второй эры закончилась война Заклятия. В Высших сферах Адон победил кровавого Гифона, в Митгаре Модру потерпел сокрушительное поражение от Великого Союза. Затем Адон наложил заклятие на всех обитателей Унтергарда, которые сражались в этой войне на стороне Гифона: они были прокляты навеки и солнечный свет отныне стал для них смертельным. Сам же Гифон был низвергнут за пределы Сфер. Так закончилась Вторая эра и началась Третья, и жизнь шла своим чередом до Четвертой эры. ',
 'Дубчековцы — приверженцы Александра Дубчека, чехословацкого государственного и общественного деятеля, главного инициатора курса реформ, начатых в 1968 году, известных как Пражская весна. Во времена горбачевской перестройки в СССР он стал одним из активнейших участников Бархатной революции в Чехословакии.',
 'Так. Эти руны мне уже знакомы. Начало этой истории лежит еще в Первой Эпохе. Девять Учеников Мелькора получили их, и один, видимо, забрел далеко на восток… Что все же означали эти талисманы и руны? Зач

In [121]:
new_data = new_data.loc[new_data.description.str.len() > 600]
new_data.shape

(2023, 6)

In [124]:
search = ['примечания', 'прим.', 'приложение', 'автор', 'заметк', 'предисл', 'замечан', 'переводчик', 'послесловие' ]
new_data = new_data.loc[~new_data.name.str.lower().str.contains('|'.join(search))]

In [125]:
def del_symbols(data):
    data.description = data.description.str.replace('\xa0', ' ').replace('\n', ' ').replace('\r', '').replace('\xad', '')
    return data

In [ ]:
new_data = del_symbols(new_data)

In [127]:
list(new_data.description.sample(5))

['Тайный язык — см. чаккур. . Такерби Андербэнк — варорец из Боскиделла, муж Меррили, отец Рейвен. Терновый стрелок. Герой Зимней войны, автор Книги Рейвен. . Таларин — эльфийский правитель, муж Раэли, отец Гилдора, Ванидора и Феон. . Тамбл — река, протекающая с севера на юг, от Гримволлских гор через Арденскую долину и впадающая в реку Кейр. . Твилл — древний язык варорцев. . Темное море — Черное озеро в долине Врат, обиталище Кракена. . Темный Шпиль — одна из четырех вершин Куадрана, под которым расположен Крагген–кор. Черного цвета. На ее западном склоне находятся Закатные Врата. . Терновник — колючий кустарник, достигает пятидесяти метров в высоту. Растет в речных долинах на границах Боскиделла. . Терновые стрелки — варорские отряды, охраняющие границы Боскиделла в неспокойные времена. . Тиллок — варорская деревня в Истаделле в Боскиделле. . Тимбл — деревушка в Боскиделле. . Тобин Форгфаер — гном из клана Дьюрека, один из участников краггенкорского отряда. Был ранен в ногу в битве 

In [128]:
# new_data.to_pickle('datas/fantasy_data.xz')

# Import pickle

In [6]:
# new_data = pd.read_pickle('datas/fantasy_data.xz')

In [129]:
len(new_data.book_title.unique())

85

In [133]:
new_data = new_data.drop(columns=['from_doc', 'book_path'])

In [ ]:
'''
Код ниже разбивает текст по предложениям на фрагменты меньше 2000 символов, тк последовательности все равно будут обрезаны, 
а мне бы не хотелось терять информацию или обрезать её бездумно
'''

In [ ]:
new_lst = []
start = time.time()
for author, book_name, ch_name, descr in new_data.values:
    print(f'work with {descr}')
    descr_lst = re.findall('[^.]+\.', descr)
    i = 1
    flag = True
    seq_num = 0
    while len(descr_lst) > 0:
        print(f'New iter {time.time() - start}: len(descr_lst > 0), {len(descr_lst)}')

        if flag:
            ind_1 = 0
            ind_2 = 11

        join_descr = ''.join(descr_lst[ind_1:ind_2])
        if len (join_descr) > 2000:
            ind_2 -= i
            if ind_2 == 0:
                print('ind_2 == 0')
                break
                
            flag = False
        else:
            # Добавляю в дикт новый кусочек, удаляю добавленные посл-сти из листа
            temp_data = pd.DataFrame({'author': [author], 'book_title': [book_name], 'name': [ch_name], 'seq_num': [seq_num], 'description': [join_descr]})
            new_lst.append(temp_data)
            del descr_lst[ind_1:ind_2]

            flag = True
            seq_num += 1
            print(f'Добавил посл-сть {seq_num} из главы {ch_name} {book_name}')

In [135]:
final_data = pd.concat(new_lst)

In [136]:
# final_data.to_pickle('datas/fant_fin_data.xz') 

In [137]:
final_data.description.str.len().describe()

count    111022.000000
mean        787.056466
std         282.084536
min           7.000000
25%         591.000000
50%         751.000000
75%         943.000000
max        2000.000000
Name: description, dtype: float64

In [138]:
final_data= final_data.loc[final_data.description.str.len() > 100]
list(final_data.description.sample(5))

[' – Что ты здесь делаешь? — сурово спросил Гэндальф. — Разве ты не знаешь, что воинам Цитадели нельзя покидать крепость без разрешения Правителя?. – Он разрешил, — ответил Пиппин. — Он отослал меня. Но я боюсь. Там может случиться что–то страшное. Кажется, мой Правитель лишился рассудка. Я боюсь, что он убьет и себя и Фарамира. Гэндальф! Сделай что–нибудь. Маг взглянул в зияющий проем Ворот, прислушался к шуму битвы на равнине. – Мое место там, — сказал он.',
 ' Не без причины лес звался Древлепущей. Он и правда был древен, один из последних островов некогда бескрайнего леса, хранящий в чаще праотцев деревьев, стареющих не быстрее окрестных холмов и помнящих былые эпохи своего безраздельного владычества. Бесчисленные годы дали им спесивую корневую мудрость и напитали злобой. Но самым опасным из них был Серый Лох. Сердце его насквозь потемнело, а сила была зелена. Он управлял ветрами, простирая свои мыслепесни по обе стороны реки. Его древний неуемный дух до тех пор тянул из земли силу

In [139]:
patt_bracketrd = re.compile(r'\([^()]*\)') #Убирает всё, что в круглых скобках "()".
patt_bracketsq = re.compile(r'\[[^\[\]]*\]') #Убирает всё, что в квадратных скобках "[]".
patt_list = [patt_bracketrd, patt_bracketsq]
for patt in patt_list:
    final_data.description = final_data.description.str.replace(patt, '', regex=True)

In [140]:
def check_me(texts):
    temp_list = []
    texts = re.findall('[^.]+\.', texts)
    for text in texts:
        text = text.lower()
        if len(text) > 4:
            if len(text.split()) > 1:
                if not re.search(r'[A-Za-z]', text):
                    temp_list.append(text)
    return ''.join(temp_list)

In [141]:
final_data.description = final_data.description.apply(check_me)

In [142]:
final_data.book_title.unique().shape

(85,)

In [145]:
final_data = final_data.drop('seq_num', axis='columns')

In [146]:
final_data.to_pickle('datas/fant_fin_data.xz')